In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from treehopper.hoppers import hopper, treehopper, RPartition, PCATreePartition
import treehopper.explore as e
from treehopper.utils import haus
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
from ipywidgets import interact
from sklearn.preprocessing import LabelEncoder
from copy import deepcopy
import pandas as pd

In [ ]:
adata = sc.read_h5ad("/home/yilin/PCA/epi.h5ad")

In [ ]:
rgauss = adata.X

In [ ]:
coord = pd.read_csv('/home/yilin/PCA/results/X_umap_aucell.csv',index_col=0)

In [ ]:
leiden = pd.read_csv('/home/yilin/PCA/leiden.csv')

In [ ]:
adata.obsm['X_umap']=coord.values

In [ ]:
adata.obs['leiden'] = leiden.leiden.values

In [ ]:
h = treehopper(rgauss, partition=PCATreePartition(rgauss),max_partition_size=10, splits=float('inf'))
h.hop(2300)
epi_hopper = adata[h.path]

In [ ]:
## cell index
len(h.path)

In [ ]:
pd.DataFrame(h.path).to_csv('hopper_cell_index.csv')

In [ ]:
epi_hopper = adata[h.path]
epi_hopper.write_h5ad('epi_hopper.h5ad',compression='gzip')

In [ ]:
epi_hopper = sc.read_h5ad('epi_hopper.h5ad')

In [ ]:
## generate 10 sketches of size 2300
for i in range(10):
    file_name = 'epi_hopper'+str(i)+'.h5ad'
    h = treehopper(rgauss, partition=PCATreePartition(rgauss),max_partition_size=10, splits=float('inf'))
    h.hop(2300)
    epi_hopper = adata[h.path]
    epi_hopper.write_h5ad(file_name,compression='gzip')

In [ ]:
cluster = adata.obs['leiden'].astype(int)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(10,10))
ax.scatter(adata.obsm['X_umap'][:,0],adata.obsm['X_umap'][:,1],c=cluster, cmap='tab20',s=5,alpha=0.8)
ax.scatter(epi_hopper.obsm['X_umap'][:,0],epi_hopper.obsm['X_umap'][:,1],color='red',s=5,zorder=3)
ax.axis('off')
plt.show()